In [2]:
#libraries
import numpy as np
import pandas as pd
import math 
import itertools
import sklearn
import sklearn.metrics
import sklearn.model_selection as sms
import matplotlib
import seaborn as sns

import keras.models
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.utils import to_categorical
import tensorflow as tf


In [3]:
np.random.seed(69)

In [4]:
#Fashion MNIST data
train = pd.read_csv("fashion-mnist_train.csv")
test = pd.read_csv("fashion-mnist_test.csv")
train_set = train.copy()
test_set = test.copy()
train_set.head()
print(train_set.shape)
print(test_set.shape)

(60000, 785)
(10000, 785)


In [5]:
train_set.label.unique()

array([2, 9, 6, 0, 3, 4, 5, 8, 7, 1])

In [6]:
#reshape and classify
def data_preprocessing(raw):
    label = tf.keras.utils.to_categorical(raw.label, 10)
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, 28, 28, 1)
    image = x_shaped_array / 255
    return image, label

X, y = data_preprocessing(train_set)
X_test, y_test = data_preprocessing(test_set)

In [7]:
#validation
X_train, X_val, y_train, y_val = sms.train_test_split(X, y, test_size=0.25, random_state=69)

In [8]:
#cnn hyperparameters from kaggle: 
#https://www.kaggle.com/code/rutvikdeshpande/fashion-mnist-cnn-beginner-98 

model = tf.keras.Sequential()

# first layer
model.add(Conv2D(32, (3,3), padding='same', input_shape=(28,28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# second layer
model.add(Conv2D(64, (3,3), padding='same', activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# Fully connected layer 
model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))
model.add(BatchNormalization())

# Output layer 
model.add(Dense(10, activation=tf.nn.softmax))

Metal device set to: Apple M1 Max


2023-11-11 15:47:15.467285: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-11 15:47:15.467450: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                        

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [11]:
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [12]:
#data augmentation from https://www.kaggle.com/code/fuzzywizard/fashion-mnist-cnn-keras-accuracy-93/notebook

datagen = ImageDataGenerator(
        rotation_range = 8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        shear_range = 0.3,# shear angle in counter-clockwise direction in degrees  
        width_shift_range=0.08,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.08,  # randomly shift images vertically (fraction of total height)
        vertical_flip=True)  # randomly flip images

In [13]:
#training

In [14]:
batch_size = 128
epochs = 100

history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size), epochs = epochs, 
                              validation_data = (X_val, y_val), verbose=2, 
                              steps_per_epoch=X_train.shape[0] // batch_size,
                              callbacks = [reduce_lr])

/var/folders/vt/50rt81d15835tww3hmfy4dl00000gn/T/ipykernel_35665/3608375265.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size), epochs = epochs,


Epoch 1/100


2023-11-11 15:47:15.710484: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-11 15:47:16.093791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-11 15:47:22.284287: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


351/351 - 10s - loss: 0.6443 - accuracy: 0.7621 - val_loss: 2.7942 - val_accuracy: 0.2793 - lr: 0.0010 - 10s/epoch - 29ms/step
Epoch 2/100
351/351 - 9s - loss: 0.4790 - accuracy: 0.8220 - val_loss: 0.4222 - val_accuracy: 0.8472 - lr: 9.0000e-04 - 9s/epoch - 26ms/step
Epoch 3/100
351/351 - 9s - loss: 0.4269 - accuracy: 0.8413 - val_loss: 0.3745 - val_accuracy: 0.8612 - lr: 8.1000e-04 - 9s/epoch - 26ms/step
Epoch 4/100
351/351 - 9s - loss: 0.4026 - accuracy: 0.8495 - val_loss: 0.4571 - val_accuracy: 0.8186 - lr: 7.2900e-04 - 9s/epoch - 26ms/step
Epoch 5/100
351/351 - 9s - loss: 0.3769 - accuracy: 0.8597 - val_loss: 0.3270 - val_accuracy: 0.8799 - lr: 6.5610e-04 - 9s/epoch - 26ms/step
Epoch 6/100
351/351 - 9s - loss: 0.3631 - accuracy: 0.8655 - val_loss: 0.3068 - val_accuracy: 0.8891 - lr: 5.9049e-04 - 9s/epoch - 26ms/step
Epoch 7/100
351/351 - 9s - loss: 0.3492 - accuracy: 0.8713 - val_loss: 0.3074 - val_accuracy: 0.8877 - lr: 5.3144e-04 - 9s/epoch - 27ms/step
Epoch 8/100
351/351 - 9s - 

Epoch 59/100
351/351 - 9s - loss: 0.2534 - accuracy: 0.9053 - val_loss: 0.2518 - val_accuracy: 0.9100 - lr: 2.2185e-06 - 9s/epoch - 26ms/step
Epoch 60/100
351/351 - 9s - loss: 0.2518 - accuracy: 0.9072 - val_loss: 0.2504 - val_accuracy: 0.9104 - lr: 1.9967e-06 - 9s/epoch - 26ms/step
Epoch 61/100
351/351 - 9s - loss: 0.2517 - accuracy: 0.9061 - val_loss: 0.2487 - val_accuracy: 0.9115 - lr: 1.7970e-06 - 9s/epoch - 26ms/step
Epoch 62/100
351/351 - 9s - loss: 0.2554 - accuracy: 0.9058 - val_loss: 0.2492 - val_accuracy: 0.9106 - lr: 1.6173e-06 - 9s/epoch - 26ms/step
Epoch 63/100
351/351 - 9s - loss: 0.2528 - accuracy: 0.9055 - val_loss: 0.2512 - val_accuracy: 0.9100 - lr: 1.4556e-06 - 9s/epoch - 26ms/step
Epoch 64/100
351/351 - 9s - loss: 0.2554 - accuracy: 0.9057 - val_loss: 0.2506 - val_accuracy: 0.9103 - lr: 1.3100e-06 - 9s/epoch - 26ms/step
Epoch 65/100
351/351 - 9s - loss: 0.2503 - accuracy: 0.9067 - val_loss: 0.2494 - val_accuracy: 0.9109 - lr: 1.1790e-06 - 9s/epoch - 26ms/step
Epoch 

In [16]:
#test model
score = model.evaluate(X_test,y_test)
print("Loss: {:.4f}".format(score[0]))
print("Accuracy: {:.4f}".format(score[1]))

313/313 [==============================] - 3s 9ms/step - loss: 0.2357 - accuracy: 0.9127
Loss: 0.2357
Accuracy: 0.9127


In [17]:
model.save('ecoThreads_classification_model.keras')